In [1]:
from utils import *

In [23]:
df = pd.read_csv('test_data.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df['tradedate'] = pd.to_datetime(df['tradedate'])
df.set_index(['tradedate','wind_code'], inplace=True)
grouper = ['tradedate']

In [119]:
def resample(data: pd.DataFrame, freq: str = 'M'):
    date_idx = pd.DatetimeIndex(data.index.get_level_values('tradedate').unique())
    year = date_idx.year
    if freq == 'D':
        freq = date_idx.dayofyear
    elif freq == 'Y':
        freq = date_idx.year
    elif freq == 'M':
        freq = date_idx.month
    elif freq == 'W':
        year = date_idx.isocalendar().year  #每年的最后一天不一定是当年（有可能是下一年）的星期，最开始一天不一定是当年（有可能是上一年）的星期
        freq = date_idx.isocalendar().week
    elif freq == 'Q':
        freq = date_idx.quarter
    date_idx = date_idx.to_frame().groupby([year, freq], as_index=False).last().set_index('tradedate')

    return data.reindex(date_idx.index, level='tradedate')    

In [42]:
def df_qcut(df: pd.DataFrame, n_groups: int = 5):
        if df.empty or df.count().sum() == 0:
            return pd.DataFrame().reindex_like(df)

        def q_cut(x: pd.Series, n_groups: int = n_groups):
            try:
                return pd.qcut(x, q=n_groups, duplicates='drop', labels=['p' + str(i) for i in range(1, n_groups + 1)])
            except:
                try:
                    return pd.qcut(x.rank(method='first'),
                                   q=np.linspace(0, 1, num=n_groups + 1),
                                   duplicates='drop',
                                   labels=['p' + str(i) for i in range(1, n_groups + 1)])
                except:
                    try:
                        return pd.qcut(x.rank(method='first'), q=n_groups, duplicates='drop', labels=['p' + str(i) for i in range(1, n_groups + 1)])
                    except ValueError as e:
                        _, n_bins = pd.qcut(x.rank(method='first'), q=n_groups, duplicates='drop', retbins=True, labels=False)
                        if len(n_bins) > 0:
                            return pd.qcut(x.rank(method='first'),
                                           q=n_groups,
                                           duplicates='drop',
                                           labels=['p' + str(i) for i in range(1, len(n_bins))])
                        else:
                            print(x)
                            raise ValueError(len(x.unique()), repr(e))

        return df.apply(q_cut)

In [170]:
test_1 = df.swaplevel()

avgPrice  changeRatio      volume
wind_code tradedate                                     
000001.SZ 2023-04-03  12.618163     1.117318  74042886.0
          2023-04-04  12.629816    -0.157853  64745722.0
          2023-04-06  12.541345    -0.553360  56332130.0
          2023-04-07  12.622519     0.317965  60792526.0
          2023-04-10  12.651768     0.475436  64018181.0
...                         ...          ...         ...
689009.SH 2023-07-21  34.339458    -0.204141   2493386.0
          2023-07-24  33.477977    -2.074810   3802906.0
          2023-07-25  34.094390     2.059087   2908621.0
          2023-07-26  33.863879    -1.286550   2169792.0
          2023-07-27  33.836138    -0.740521   2835869.0

[387759 rows x 3 columns]

In [169]:
test_2

avgPrice volume
wind_code tradedate                 
000001.SZ 2023-04-28       p3     p5
          2023-05-31       p3     p5
          2023-06-30       p3     p5
          2023-07-27       p3     p5
000002.SZ 2023-04-28       p3     p5
...                       ...    ...
688981.SH 2023-07-27       p5     p5
689009.SH 2023-04-28       p5     p2
          2023-05-31       p5     p2
          2023-06-30       p5     p4
          2023-07-27       p5     p2

[19940 rows x 2 columns]

In [167]:
A = []
factor_date_grouped = test_1.groupby(level='wind_code')
for name, group in factor_date_grouped:
    date_idx_group = pd.DatetimeIndex(group.index.get_level_values('tradedate').unique())
    new_inde
    group = group.reindex(date_idx_group, method='ffill')
    A.append(group)

TypeError: Cannot compare dtypes object and datetime64[ns]